In [114]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from joblib import dump, load

In [115]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.Churn.value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

In [116]:
df = df.drop(columns=['customerID'])

In [117]:
# df['TotalCharges'] = df['TotalCharges'].astype(float)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [118]:
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})

In [119]:
df.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [120]:
# df[df.isna().any(axis=1)]
df = df.dropna()

In [121]:
X = df.drop(columns=['Churn'])
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [123]:
# Balance test data
assert np.any(X_test.index == y_test.index), "Indexes in X_test do not match with y_test, cannot continue"

n = min(y_test.value_counts().values)
y_test_groups = []
for target in y_test.unique():
    if len(y_test[y_test == target]) == n:
        y_test_groups.append(y_test[y_test == target])
    else:
        y_test_groups.append(y_test[y_test == target].sample(n=n, random_state=42))
y_test = pd.concat(y_test_groups)
X_test = X_test.loc[y_test.index]

assert np.any(X_test.index == y_test.index), "Indexes in X_test do not match with y_test"

In [124]:
# Balance train data
assert np.any(X_train.index == y_train.index), "Indexes in X_train do not match with y_train, cannot continue"

n = min(y_train.value_counts().values)
y_train_groups = []
for target in y_train.unique():
    if len(y_train[y_train == target]) == n:
        y_train_groups.append(y_train[y_train == target])
    else:
        y_train_groups.append(y_train[y_train == target].sample(n=n, random_state=42))
y_train = pd.concat(y_train_groups)
X_train = X_train.loc[y_train.index]

assert np.any(X_train.index == y_train.index), "Indexes in X_train do not match with y_train"

In [125]:
y_train.value_counts()

Churn
1    1495
0    1495
Name: count, dtype: int64

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
2182,Male,0,Yes,Yes,56,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,103.20,5873.75
4387,Male,0,No,Yes,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Mailed check,20.25,20.25
2870,Male,0,Yes,No,57,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Two year,No,Electronic check,107.95,5969.85
2288,Female,0,No,No,5,Yes,No,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Mailed check,85.30,420.45
960,Male,1,Yes,No,7,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Bank transfer (automatic),29.80,220.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,Female,0,No,No,1,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Bank transfer (automatic),73.50,73.50
4444,Female,0,No,No,3,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.85,63.75
2869,Female,0,No,No,6,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,91.10,455.30
5001,Male,0,No,No,19,Yes,No,DSL,No,No,Yes,Yes,No,No,Month-to-month,Yes,Credit card (automatic),55.00,1046.50


In [34]:
y_test.value_counts()

Churn
0    1033
1     374
Name: count, dtype: int64

In [126]:
categorical_columns = []
for i in df.columns:
    if df[i].dtype == 'object':
        categorical_columns.append(i)

In [127]:
dump(categorical_columns, '../models/categorical_columns.joblib')

['../models/categorical_columns.joblib']

In [128]:
ordinal = OrdinalEncoder()

In [129]:
ordinal.fit(X_train[categorical_columns])

X_train[categorical_columns] = ordinal.transform(X_train[categorical_columns])

In [130]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)

In [131]:
model = LogisticRegression()

In [132]:
model.fit(X_train, y_train)

LogisticRegression()

In [133]:
dump(ordinal, '../models/Ordinal_Encoder.joblib')
dump(scaler, '../models/Standard_Scaler.joblib')
dump(model, '../models/Logistic_Regression.joblib')

['../models/Logistic_Regression.joblib']

In [134]:
type(X_train)

numpy.ndarray

# Eval

In [135]:
scaler = load('../models/Standard_Scaler.joblib')
ordinal = load('../models/Ordinal_Encoder.joblib')
model = load('../models/Logistic_Regression.joblib')
categorical_columns = load('../models/categorical_columns.joblib')

In [136]:
X_test[categorical_columns] = ordinal.transform(X_test[categorical_columns])

In [137]:
X_test = scaler.transform(X_test)

In [138]:
y_pred = model.predict(X_test)

In [139]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7580213903743316


In [25]:
print("metrix:", confusion_matrix(y_test, y_pred))

metrix: [[920 113]
 [189 185]]


In [140]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [141]:
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

In [142]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [143]:
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.72


In [144]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.75      0.67      0.71       374
           1       0.70      0.77      0.74       374

    accuracy                           0.72       748
   macro avg       0.73      0.72      0.72       748
weighted avg       0.73      0.72      0.72       748



In [145]:
param_grid = {
    'max_depth': [3, 4, 5, 7, 8],
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
    'n_estimators': [50, 100, 200, 300, 400, 500]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss'),
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=3,
                           verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
Best parameters found:  {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 50}


In [146]:
y_pred = grid_search.best_estimator_.predict(X_test)

In [147]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.76


# Refactorig

In [31]:
from typing import Tuple, Dict

In [32]:
def evaluation(model: xgb.XGBRegressor, X_test: np.ndarray, y_test: np.ndarray) -> Dict[str, float]:
    y_pred_test = model.predict(X_test)
    return {
        'accuracy': accuracy_score(y_test, y_pred_test)
    }

In [37]:
from typing import Tuple, Dict, Union


MODEL_PATH = '../models/'

def read_prepare_df(PATH: str) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    df = pd.read_csv(PATH)

    df = df.drop(columns=['customerID'])
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})

    df = df.dropna()
    X = df.drop(columns=['Churn'])
    y = df['Churn']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test


# def data_preprocessing(X_train: pd.DataFrame) -> np.ndarray:
#     categorical_columns = X_train.select_dtypes(include=['object']).columns

#     dump(categorical_columns, MODEL_PATH + 'categorical_columns.joblib')

#     ordinal = OrdinalEncoder()
#     ordinal.fit(X_train[categorical_columns])

#     X_train[categorical_columns] = ordinal.transform(X_train[categorical_columns])

#     scaler = StandardScaler()
#     scaler.fit(X_train)

#     X_train = scaler.transform(X_train)

#     return X_train


def model_train(model: xgb.XGBRegressor, X_train: np.ndarray,
           X_test: np.ndarray, y_train: np.ndarray, y_test: np.ndarray) -> Dict[str, float]:
    
    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
    model.fit(X_train, y_train)

    dump(model, MODEL_PATH + 'Logistic_Regression.joblib')


    # dump(model, MODEL_PATH + 'XGBoost_model.joblib')
    param_grid = {
        'max_depth': [3, 4, 5, 7, 8],
        'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
        'n_estimators': [50, 100, 200, 300, 400, 500]
    }

    # Create a GridSearchCV object
    grid_search = GridSearchCV(estimator=xgb.XGBClassifier(
            objective='binary:logistic', eval_metric='logloss'),
            param_grid=param_grid,
            scoring='accuracy',
            cv=3,
            verbose=1
        )

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # Print the best parameters
    print("Best parameters found: ", grid_search.best_params_)
    
    dump(grid_search, MODEL_PATH + 'XGBoost_classifier.joblib')

    return evaluation(grid_search, X_test, y_test)

In [38]:
ScalerType = Union[StandardScaler, MinMaxScaler]

DATA_PATH = '../data/'

def ordinal_encoding(X: pd.DataFrame, y: pd.Series) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    ordinal = OrdinalEncoder()
    categorical_columns = X_train.select_dtypes(include=['object']).columns

    ordinal.fit(X_train[categorical_columns])
    X_train[categorical_columns] = ordinal.transform(X_train[categorical_columns])
    X_test[categorical_columns] = ordinal.transform(X_test[categorical_columns])

    dump(ordinal, MODEL_PATH + 'Ordinal_Encoder.joblib')
    dump(X.columns, MODEL_PATH + 'columns.joblib')

    return X_train, X_test, y_train, y_test


def standardizing(X_train: pd.DataFrame, X_test: pd.DataFrame, scaler: ScalerType) -> Tuple[np.ndarray, np.ndarray]:
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    dump(scaler, MODEL_PATH + 'Standard_Scaler.joblib')

    return X_train, X_test


def encode_and_update(data: pd.DataFrame, ordinal_path: str) -> OrdinalEncoder:
    ordinal = load(ordinal_path)
    categorical_columns = data.select_dtypes(include=['object']).columns

    for index, col in enumerate(categorical_columns):
        unique_items = set(data[col])
        known_items = set(ordinal.categories_[index])
        new_items = unique_items - known_items

        if new_items:
            ordinal.categories_[index] = np.append(ordinal.categories_[index], list(new_items))

    dump(ordinal, MODEL_PATH + 'Ordinal_Encoder.joblib')
    return ordinal


def load_joblibs() -> Tuple[pd.Index, OrdinalEncoder, ScalerType, xgb.XGBRegressor]:
    cols = load(MODEL_PATH + 'columns.joblib')
    ordinal = load(MODEL_PATH + 'Ordinal_Encoder.joblib')
    standard = load(MODEL_PATH + 'Standard_Scaler.joblib')
    model = load(MODEL_PATH + 'modXGBoost_classifierel.joblib')

    return cols, ordinal, standard, model

In [42]:
from datetime import datetime


def make_predictions(data: pd.DataFrame) -> pd.DataFrame:
    time = datetime.today()
    cols, ordinal, standard, model = load_joblibs()

    ids = data['customerID']
    df_test = data[cols]
    ordinal = encode_and_update(df_test, MODEL_PATH + 'Ordinal_Encoder.joblib')
    df_test[df_test.select_dtypes(include=['object']).columns] = ordinal.transform(
        df_test.select_dtypes(include=['object']))

    df_test = standard.transform(df_test)

    y_pred = model.predict(df_test)
    submission_df = pd.DataFrame({'customerID': ids, 'Churn': y_pred})
    submission_df.to_csv(DATA_PATH + f'prediction_{time}.csv', index=False)

    print(f"Submission file created successfully.\nDateTime: {time}")

    return submission_df

# testing

In [3]:
import sys
import os

# sys.path.append('..')
sys.path.append(os.path.abspath('../code/model_pipeline'))

In [6]:
import pandas as pd
from train import model_train

# training_data_df = pd.read_csv('../data/train.csv')
model_performance_dict = model_train('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(model_performance_dict)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
Best parameters found:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
{'model performance before tuning': {'accuracy': '0.80'}}


In [4]:
from inference import make_predictions

print(make_predictions(df))

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- TotalCharges


In [4]:
cols = load('../models/columns.joblib')

In [5]:
cols

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')

In [7]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object